# **Notebook for DV101 Capstone Project**

In [3]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


## Tasks 1. Starting the Project

In [4]:
#Importing libraries
import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
import requests

In [5]:
# !pip install bs4
# !pip install geocoder
# !pip install geopy

## Task 2. Getting raw data from Wikipedia

In [6]:
#Parsering data from Wikipedia
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html = urllib.request.urlopen(url).read()
soup = BeautifulSoup(html, 'html.parser')

#Extracting the table with required information based on HTML DOM
table = soup.findAll("table", {'class': "wikitable sortable"})

## Task 3. Transforming raw data to the Dataframe

In [7]:
#Converting to dataframe with added columns and dropping indexes
data = pd.read_html(str(table), flavor='html5lib')
df=pd.DataFrame(np.concatenate(data), columns=['Postal Code', 'Borough', 'Neighborhood'])
# df.set_index('Postal Code', inplace=True)

#Dropping 'Not assigned' postcodes through value replacement
df.replace('Not assigned', np.nan, inplace = True)
df = df.dropna()
df.reset_index(drop=True, inplace=True)
df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [8]:
df.shape

(103, 3)

### 10 marks for the Dataframe with three columns

## Task 4. Getting geocoordinates

Geocode library doesn't work due to consistently sending empty data. That's why CSV file was used.

In [9]:
# import geocoder # import geocoder

# # initialize your variable to None
# lat_lng_coords = None

# for postal_code in df['Postal Code']:
#     # loop until you get the coordinates
#     while(lat_lng_coords is None):
#         g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
#         lat_lng_coords = g.latlng
#     latitude, longitude = lat_lng_coords
#     df['latitude'] = latitude
#     df['longitude'] = longitude
# df.head()

In [10]:
url_csv = 'http://cocl.us/Geospatial_data'
lat_lng_coords = pd.read_csv(url_csv)
df_coords = pd.DataFrame(lat_lng_coords)
df_coords.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [11]:
neighborhoods = df.merge(df_coords, on='Postal Code', sort=True)
neighborhoods.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [12]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


### 2 marks for the Dataframe with geo_coordinates

## Task 5. Segmentation and clustering

Create a map of Toronto with neighborhoods superimposed on top.

In [13]:
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import folium
%matplotlib inline 
from sklearn.cluster import KMeans 

In [14]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

address = 'Toronto, Ontario, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [15]:
tor_map = folium.Map(location=[latitude+0.05, longitude], zoom_start=10.5)

post_codes_markers = folium.map.FeatureGroup()

for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='yellow',
        fill=True,
        fill_color='blue',
        fill_opacity=0.6,
        parse_html=False).add_to(tor_map) 
   
tor_map

Define Foursquare Credentials and Version

In [16]:
CLIENT_ID = 'AWIJURFRIC1ZUF12TMFZGCMQP4D1RXDF5OH5RCDCCVMPINVF' # your Foursquare ID
CLIENT_SECRET = 'VDJ5K0TBKRKPXJRM4PM33FP0JUIWOXRJ0IG2PRCNYI50VV3F' # your Foursquare Secret
ACCESS_TOKEN = 'BUNJF3OQFCFMUOVHOUHHFBCL34L5EL2GVF1SDSC3ZKJXCAMO' # your FourSquare Access Token
VERSION = '20210101'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: AWIJURFRIC1ZUF12TMFZGCMQP4D1RXDF5OH5RCDCCVMPINVF
CLIENT_SECRET:VDJ5K0TBKRKPXJRM4PM33FP0JUIWOXRJ0IG2PRCNYI50VV3F


Let's explore the first neighborhood in our dataframe.

In [17]:
neighborhoods.loc[0, 'Neighborhood']

'Malvern, Rouge'

Get the neighborhood's latitude and longitude values.

In [18]:
neighborhood_latitude = neighborhoods.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = neighborhoods.loc[0, 'Longitude'] # neighborhood longitude value
neighborhood_name = neighborhoods.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Malvern, Rouge are 43.806686299999996, -79.19435340000001.


 Let's get the top 100 venues that are within a radius of 500 meters.

In [19]:
radius = 500
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&radius={}&limit={}'.format(
    CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, radius, LIMIT)

results = requests.get(url).json()

In [20]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories'][0]
    except:
        categories_list = row['categories']
    
    if len(categories_list) == 0:
        return None
    else:
        return categories_list['name']
    
    
venues = results['response']['venues']
    
nearby_venues = pd.json_normalize(venues) # flatten JSON

# # filter columns
filtered_columns = ['name', 'categories', 'location.lat', 'location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# # filter the category for each row
nearby_venues['categories'] = nearby_venues.apply(get_category_type, axis=1)

# # # clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,City Hall Council Chambers,City Hall,43.651827,-79.383949
1,Toronto City Hall,City Hall,43.653140,-79.383967
2,City of Toronto Civic Innovation Office,City Hall,43.653454,-79.383952
3,City Hall Podium Green Roof,Garden,43.653504,-79.383866
4,Kew Gardens Playground,Playground,43.653225,-79.383185


In [21]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

67 venues were returned by Foursquare.


### Let's explore Neighborhoods in Toronto

In [22]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])


    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [23]:
toronto_venues = getNearbyVenues(neighborhoods['Neighborhood'], neighborhoods['Latitude'], neighborhoods['Longitude'])

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale, Willowdale East
York Mills West
Willowdale, Willowdale West
Parkwoods
Don Mills
Don Mills
Bathurst Manor, Wilson Heights, Downsview North
Northwood Park, York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Parkview Hill, Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto, Broadview North (Old East York)
The Danforth West, 

In [24]:
print(toronto_venues.shape)
toronto_venues.head()

(2099, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Malvern, Rouge",43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Chris Effects Painting,43.784343,-79.163742,Construction & Landscaping
3,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
4,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Affordable Toronto Movers,43.787919,-79.162977,Moving Target


In [25]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Alderwood, Long Branch",7,7,7,7,7,7
"Bathurst Manor, Wilson Heights, Downsview North",22,22,22,22,22,22
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",24,24,24,24,24,24
...,...,...,...,...,...,...
"Willowdale, Willowdale East",34,34,34,34,34,34
"Willowdale, Willowdale West",5,5,5,5,5,5
Woburn,4,4,4,4,4,4


In [26]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 270 uniques categories.


### Let's analyze each Neighborhood in Toronto

In [27]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.shape

(2099, 270)

In [28]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,"Willowdale, Willowdale East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.029412,0.0,0.0,0.0,0.0,0.0
92,"Willowdale, Willowdale West",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,0.0
93,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.000000,0.0,0.0,0.0,0.0,0.0
94,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.125,0.000000,0.0,0.0,0.0,0.0,0.0


#### Let's print each neighborhood along with the top 5 most common venues

In [29]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0                     Lounge  0.25
1  Latin American Restaurant  0.25
2             Breakfast Spot  0.25
3             Clothing Store  0.25
4         Mexican Restaurant  0.00


----Alderwood, Long Branch----
          venue  freq
0   Pizza Place  0.29
1  Skating Rink  0.14
2      Pharmacy  0.14
3   Coffee Shop  0.14
4           Pub  0.14


----Bathurst Manor, Wilson Heights, Downsview North----
         venue  freq
0  Coffee Shop  0.09
1         Bank  0.09
2  Pizza Place  0.05
3  Gas Station  0.05
4     Pharmacy  0.05


----Bayview Village----
                 venue  freq
0                 Café  0.25
1  Japanese Restaurant  0.25
2                 Bank  0.25
3   Chinese Restaurant  0.25
4                Motel  0.00


----Bedford Park, Lawrence Manor East----
                venue  freq
0           Juice Bar  0.08
1      Sandwich Place  0.08
2  Italian Restaurant  0.08
3         Coffee Shop  0.08
4    Sushi Restaurant  0.04


----Bercz

In [30]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [31]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Latin American Restaurant,Lounge,Clothing Store,Breakfast Spot,Electronics Store,Eastern European Restaurant,Ethiopian Restaurant,Dumpling Restaurant,Dessert Shop,Drugstore
1,"Alderwood, Long Branch",Pizza Place,Gym,Skating Rink,Pharmacy,Pub,Coffee Shop,Airport Gate,Event Space,Electronics Store,Eastern European Restaurant
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Gift Shop,Health Food Store,Sandwich Place,Bridal Shop,Diner,Restaurant,Deli / Bodega,Ice Cream Shop
3,Bayview Village,Japanese Restaurant,Café,Bank,Chinese Restaurant,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Women's Store
4,"Bedford Park, Lawrence Manor East",Sandwich Place,Coffee Shop,Juice Bar,Italian Restaurant,Hobby Shop,Pub,Café,Sushi Restaurant,Indian Restaurant,Restaurant


### Cluster Neighborhoods

In [99]:
kclusters = 7

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([6, 2, 6, 6, 6, 6, 6, 6, 4, 6], dtype=int32)

In [100]:
neighborhoods_venues_sorted.drop(['Cluster Labels'], axis=1, inplace=True)

In [101]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
# neighborhoods_venues_sorted.head()
toronto_merged = neighborhoods

toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged.dropna(inplace=True)
toronto_merged.head(10) 

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,4.0,Fast Food Restaurant,Print Shop,Women's Store,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,4.0,Bar,Moving Target,Construction & Landscaping,Women's Store,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,6.0,Restaurant,Rental Car Location,Breakfast Spot,Medical Center,Intersection,Bank,Mexican Restaurant,Electronics Store,Cosmetics Shop,Coworking Space
3,M1G,Scarborough,Woburn,43.770992,-79.216917,2.0,Coffee Shop,Pharmacy,Korean BBQ Restaurant,Women's Store,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,6.0,Gas Station,Fried Chicken Joint,Bakery,Hakka Restaurant,Athletics & Sports,Bank,Thai Restaurant,Caribbean Restaurant,Discount Store,Dim Sum Restaurant
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,5.0,Playground,Women's Store,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029,2.0,Coffee Shop,Convenience Store,Department Store,Chinese Restaurant,Hobby Shop,Women's Store,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577,4.0,Bakery,Bus Line,Soccer Field,Park,Ice Cream Shop,Bus Station,Intersection,Metro Station,Distribution Center,Discount Store
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476,4.0,Motel,American Restaurant,Women's Store,Deli / Bodega,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848,6.0,College Stadium,Café,Skating Rink,General Entertainment,Women's Store,Dog Run,Dim Sum Restaurant,Diner,Discount Store,Distribution Center


In [102]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [68]:
# toronto_merged[toronto_merged['Cluster Labels'] == 2]

In [103]:
toronto_merged_grouped = toronto_merged.groupby('Cluster Labels').count().reset_index()
toronto_merged_grouped.head(kclusters)

,Cluster Labels,Postal Code,Borough,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0.0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
1,1.0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
2,2.0,16,16,16,16,16,16,16,16,16,16,16,16,16,16,16
3,3.0,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5
4,4.0,26,26,26,26,26,26,26,26,26,26,26,26,26,26,26
5,5.0,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
6,6.0,49,49,49,49,49,49,49,49,49,49,49,49,49,49,49


### Cluster 1

In [104]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,East Toronto,0.0,Health Food Store,Pub,Trail,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Women's Store


### Cluster 2

In [105]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
94,Etobicoke,1.0,Bakery,Women's Store,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Falafel Restaurant


### Cluster 3

In [106]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Scarborough,2.0,Coffee Shop,Pharmacy,Korean BBQ Restaurant,Women's Store,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
6,Scarborough,2.0,Coffee Shop,Convenience Store,Department Store,Chinese Restaurant,Hobby Shop,Women's Store,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store
10,Scarborough,2.0,Indian Restaurant,Chinese Restaurant,Pet Store,Light Rail Station,Vietnamese Restaurant,Women's Store,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
13,Scarborough,2.0,Pizza Place,Fast Food Restaurant,Convenience Store,Intersection,Italian Restaurant,Thai Restaurant,Fried Chicken Joint,Chinese Restaurant,Bank,Gas Station
15,Scarborough,2.0,Coffee Shop,Fast Food Restaurant,Pharmacy,Supermarket,Discount Store,Chinese Restaurant,Sandwich Place,Indian Restaurant,Pizza Place,Bank
24,North York,2.0,Grocery Store,Pharmacy,Pizza Place,Coffee Shop,Discount Store,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Distribution Center
29,North York,2.0,Falafel Restaurant,Coffee Shop,Caribbean Restaurant,Massage Studio,Bar,Doner Restaurant,Diner,Discount Store,Distribution Center,Dog Run
34,North York,2.0,Coffee Shop,Pizza Place,Intersection,Hockey Arena,Portuguese Restaurant,Women's Store,Distribution Center,Dessert Shop,Dim Sum Restaurant,Diner
35,East York,2.0,Pizza Place,Bank,Gym / Fitness Center,Intersection,Pharmacy,Athletics & Sports,Gastropub,Flea Market,Women's Store,Discount Store
49,Central Toronto,2.0,Light Rail Station,Coffee Shop,American Restaurant,Pizza Place,Pub,Supermarket,Bank,Fried Chicken Joint,Restaurant,Sushi Restaurant


### Cluster 4

In [111]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Scarborough,3.0,Park,Playground,Intersection,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant
21,North York,3.0,Park,Deli / Bodega,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run
40,East York,3.0,Park,Convenience Store,Intersection,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Department Store
50,Downtown Toronto,3.0,Park,Playground,Trail,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run
74,York,3.0,Park,Bar,Women's Store,Golf Course,Event Space,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop


### Cluster 5

In [112]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,4.0,Fast Food Restaurant,Print Shop,Women's Store,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant
1,Scarborough,4.0,Bar,Moving Target,Construction & Landscaping,Women's Store,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
7,Scarborough,4.0,Bakery,Bus Line,Soccer Field,Park,Ice Cream Shop,Bus Station,Intersection,Metro Station,Distribution Center,Discount Store
8,Scarborough,4.0,Motel,American Restaurant,Women's Store,Deli / Bodega,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
11,Scarborough,4.0,Sandwich Place,Middle Eastern Restaurant,Vietnamese Restaurant,Bakery,Auto Garage,Women's Store,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
17,North York,4.0,Pool,Mediterranean Restaurant,Golf Course,Fast Food Restaurant,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
23,North York,4.0,Park,Construction & Landscaping,Flower Shop,Convenience Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Donut Shop,Electronics Store,Department Store
25,North York,4.0,Park,Food & Drink Shop,Pool,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Dance Studio,Distribution Center
36,East York,4.0,Beer Store,Spa,Curling Ice,Intersection,Video Store,Bus Stop,Skating Rink,Park,Electronics Store,Eastern European Restaurant
42,East Toronto,4.0,Fast Food Restaurant,Gym,Italian Restaurant,Fish & Chips Shop,Ice Cream Shop,Restaurant,Pub,Steakhouse,Food & Drink Shop,Sushi Restaurant


### Cluster 6

In [113]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 5, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Scarborough,5.0,Playground,Women's Store,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop
48,Central Toronto,5.0,Restaurant,Playground,Women's Store,Distribution Center,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run


### Cluster 7

In [114]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 6, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Scarborough,6.0,Restaurant,Rental Car Location,Breakfast Spot,Medical Center,Intersection,Bank,Mexican Restaurant,Electronics Store,Cosmetics Shop,Coworking Space
4,Scarborough,6.0,Gas Station,Fried Chicken Joint,Bakery,Hakka Restaurant,Athletics & Sports,Bank,Thai Restaurant,Caribbean Restaurant,Discount Store,Dim Sum Restaurant
9,Scarborough,6.0,College Stadium,Café,Skating Rink,General Entertainment,Women's Store,Dog Run,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
12,Scarborough,6.0,Latin American Restaurant,Lounge,Clothing Store,Breakfast Spot,Electronics Store,Eastern European Restaurant,Ethiopian Restaurant,Dumpling Restaurant,Dessert Shop,Drugstore
18,North York,6.0,Clothing Store,Coffee Shop,Fast Food Restaurant,Restaurant,Bakery,Jewelry Store,Japanese Restaurant,Bank,Food Court,Juice Bar
19,North York,6.0,Japanese Restaurant,Café,Bank,Chinese Restaurant,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Women's Store
22,North York,6.0,Ramen Restaurant,Restaurant,Coffee Shop,Shopping Mall,Sandwich Place,Café,Pizza Place,Arts & Crafts Store,Fast Food Restaurant,Steakhouse
26,North York,6.0,Gym,Beer Store,Japanese Restaurant,Restaurant,Coffee Shop,Asian Restaurant,Italian Restaurant,Dim Sum Restaurant,Supermarket,Discount Store
27,North York,6.0,Gym,Beer Store,Japanese Restaurant,Restaurant,Coffee Shop,Asian Restaurant,Italian Restaurant,Dim Sum Restaurant,Supermarket,Discount Store
28,North York,6.0,Coffee Shop,Bank,Gift Shop,Health Food Store,Sandwich Place,Bridal Shop,Diner,Restaurant,Deli / Bodega,Ice Cream Shop


# Explanation what I decided to do and to report observations I made

I have gradually checked multiple possible number of cluster staring from 3 through 10. I have chosen number 7 because it gives the optimal distribution of neighborhood quantities among all clusters. For example, three cluster division gives (93, 1, 2) and in case of ten clusters one of them includes 82 neighborhoods. Eventually seven clasters are in the most balanced option.

As I noticed neighborhoods have been distributed into seven clusters based on patterns the most 10 common venues.